# Our models code

### installing required libraries

In [ ]:
!pip install deepface retina-face face_recognition

### importing these libraries

In [ ]:
from deepface import DeepFace
from retinaface import RetinaFace
import cv2
import pandas as pd
import face_recognition
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

### helper methods used later on

In [3]:
def age_range(age):
    if age < 0 or age > 100:
        return "Out of Range"
    else:
        return f"{int(age // 10) * 10}-{int(age // 10) * 10 + 10}"

def crop(frame, x1, y1, x2, y2):
    cropped = frame[int(y1/1.2):int(y2*1.2), int(x1/1.2):int(x2*1.2)]
    return cropped

### classifying video path that we want to analyze

#### if we want to used a surveillance camera for real time use change video capture to specified camera ex. 0

In [ ]:
video_path = "video path.mp4"
cap = cv2.VideoCapture(video_path)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

### creating fucntions for face recognition

In [ ]:
class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        images_path = glob.glob(os.path.join(images_path, "*.*"))
        print("{} encoding images found.".format(len(images_path)))
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            img_encoding = face_recognition.face_encodings(rgb_img)[0]
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names


### here is main cell for running the model

#### after done processing it is saved as seperate video

#### graph is plotted after done showing number of faces detected in each frame 

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter("output.mp4", fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
frames_processed = 1
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")
face_counts_graph = []
while (frames_processed != length):
    ret, frame = cap.read()
    print(f"Processing frame {frames_processed}/{length}")
    if not ret or cv2.waitKey(1) == ord('q'):
      print("well that's bad")
      break
    faces = RetinaFace.detect_faces(frame, threshold = 0.8)
    face_count = len(faces)
    face_counts_graph.append(face_count)
    count = 0
    for face in faces:
        x1,y1,x2,y2 = faces[face]['facial_area']
        cropped = crop(frame,x1,y1,x2,y2)
        face_locations, face_names = sfr.detect_known_faces(cropped)
        if(len(face_names)>0):
          found = face_names[0]
        else:
          found = "unknown"
        try:
          AGR = DeepFace.analyze(cropped, actions=['age', 'gender', 'race'])
          if(AGR[0]['face_confidence']>0.8):
            print(AGR[0]['face_confidence'])
            text1 = f"Conf:{AGR[0]['face_confidence']}-Age:{age_range(AGR[0]['age'])}"
            text2 = f"Gender:{AGR[0]['dominant_gender']}-race:{AGR[0]['dominant_race']}"
            cv2.rectangle(frame, (x1,y1), (x2, y2), (255, 0, 0), 4)
            cv2.putText(frame, text1,(x1, y1 - 26), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 0, 0),1)
            cv2.putText(frame, text2,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 0, 0),1)
            cv2.putText(frame, found,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 0, 0),1)
        except:
            cv2.rectangle(frame, (x1,y1), (x2, y2), (0, 0, 255), 4)
            cv2.putText(frame, f"Confidence:{round(faces[face]['score']*100,-4)}",(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 255),1)
            cv2.putText(frame, found,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 255),1)
    frames_processed += 1
    out_video.write(frame)

cap.release()
out_video.release()
cv2.destroyAllWindows()

print("Video processing complete. Output video saved as 'output.mp4'.")

plt.plot(range(len(face_counts_graph)), face_counts_graph) 

plt.xlabel("Frame Number")
plt.ylabel("Number of Faces Detected")
plt.title("Number of Faces Detected per Frame")

with open("face_counts.txt", "w") as f:
  f.write("[")
  for count in face_counts_graph:
    f.write(str(count) + ",")
  f.write("]")
plt.show()